## PsychoPy output .csv preprocessing script

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import ast

In [3]:
def chunk_list(input_list, chunk_size):
    """Splits the list into chunks of specified size."""
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]

def accuracy_with_order(user_responses, correct_answers):
    accuracies = []
    for user_resp, correct_ans in zip(user_responses, correct_answers):
        correct_count = sum(u == c for u, c in zip(user_resp, correct_ans))
        accuracy = correct_count / len(correct_ans)
        accuracies.append(accuracy)
    return accuracies

def accuracy_without_order(user_responses, correct_answers):
    accuracies = []
    for user_resp, correct_ans in zip(user_responses, correct_answers):
        correct_count = sum(user_resp.count(c) for c in set(correct_ans))
        accuracy = correct_count / len(correct_ans)
        accuracies.append(accuracy)
    return accuracies

def convert_to_floats(str_list):
    float_list = []
    for s in str_list:
        # Use ast.literal_eval to safely evaluate the string as a list
        # Then convert each element in the resulting list to a float
        floats = ast.literal_eval(s)
        float_list.extend(floats if isinstance(floats, list) else [floats])
    return float_list

# Updated function that extracts only the first float from each string
def convert_to_floats(str_list):
    float_list = []
    for s in str_list:
        # Use ast.literal_eval to safely evaluate the string as a list
        # Then convert each element in the resulting list to a float
        floats = ast.literal_eval(s)
        float_list.extend(floats if isinstance(floats, list) else [floats])
    return float_list

def convert_strings_to_float_lists(str_list):
    float_lists = []
    for s in str_list:
        if s == "None": 
            float_lists.append(s)
        else:
        # Convert the string representation of list to actual list and then to floats
            float_list = [float(x) for x in ast.literal_eval(s)]
            float_lists.append(float_list)
    return float_lists

def chunk_list(lst, n):
        chunked_list = []
        for i in range(0, len(lst), n):
            chunked_list.append(lst[i:i + n])
        return chunked_list

def has_common_letter(pair1, pair2):
        return any(letter in pair2 for letter in pair1)

# Load data

In [ ]:
pd.options.mode.chained_assignment = None
n_blocks = 12
n_supertrials = 6
n_flanker = 8
subsets = []
path = "/Users/fzaki001/Documents/wme/data/"
subjects = sorted([i[8:10] for i in os.listdir(path) if i.endswith("csv")])
wme_dict = dict({
    "sub": [],
    "acc_fl": [],
    "acc_norder": [],
    "4-1_fl_error_wm_acc_norder": [],
    "5-1_fl_error_wm_acc_norder": [],
    "4-0_fl_error_wm_acc_norder": [],
    "5-0_fl_error_wm_acc_norder": [],
    "4-many_fl_error_wm_acc_norder": [],
    "5-many_fl_error_wm_acc_norder": [],
    "4-acc_fl": [],
    "5-acc_fl": [],
    "4-acc_con": [],
    "5-acc_con": [],
    "4-acc_incon": [],
    "5-acc_incon": [],
    "1_fl_error_trials": [],
    "4-rt_con_corr": [],
    "5-rt_con_corr": [],
    "4-rt_incon_corr": [],
    "5-rt_incon_corr": [],
    "4-rt_con_incorr": [],
    "5-rt_con_incorr": [],
    "4-rt_incon_incorr": [],
    "5-rt_incon_incorr": [],
    #"pes": [],
    "4-acc_order": [],
    "5-acc_order": [],
    "4-acc_norder": [],
    "5-acc_norder": [],
    "wm_err0_fl_err0": [],
    "wm_err0_fl_err1": [],
    "wm_err1_fl_err0": [],
    "wm_err1_fl_err1": [],
    "chi2": [],
    "diff_con": [],
    "diff_incon": [],
    "4-mean_letters_from_fl_err": [],
    "5-mean_letters_from_fl_err": [],
    "4-mean_letters_from_fl_corr": [],
    "5-mean_letters_from_fl_corr": [],
    "cnt_mean": [],
    "cnt_n": [],
    "hand": [],
    "repeat": [],
    "repeat_cnt": [],
    "not_repeat_cnt": [],
    "ratio_incon_correct": [],
    "4-ratio_incon_correct": [],
    "5-ratio_incon_correct": [],
    "ratio_post_incon_correct": [],
    "4-ratio_post_incon_correct": [],
    "5-ratio_post_incon_correct": [],
    "ratio_incon_error": [],
    "4-ratio_incon_error": [],
    "5-ratio_incon_error": [],
    "ratio_post_incon_error": [],
    "4-ratio_post_incon_error": [],
    "5-ratio_post_incon_error": [],
})

for sub in subjects:
    data = pd.read_csv("{}sub-2800{}_wme-eeg_s1_r1_e1.csv".format(path, sub))
    counterbalance = (data["hand"][0], data["repeat"][0])
    row_start = list(data["conditionText"]).index(4)
    first_five_columns = data.iloc[row_start:, :5]
    columns_from_start = data.iloc[row_start:, list(data.columns).index("wm_ISI"):list(data.columns).index("supertrial_blockAcc_wm")+1]
    data = pd.concat([first_five_columns, columns_from_start], axis=1)
    print("Loading subject {}".format(sub))
    print("")

    shown_letters = data["shown_letters"].dropna().reset_index(drop=True)
    shown_letters = [i for i in shown_letters if len(i)>=4]
    for i in range(len(shown_letters)-1):
        if shown_letters[i][:4] == shown_letters[i+1][:4]:
            shown_letters[i] = shown_letters[i]+"*"
    shown_letters = [i for i in shown_letters if i[-1] != "*"]
    assert len(shown_letters) == (n_blocks * n_supertrials), "Check your data length!"
    
    wm_response = data["wm_response"].dropna().reset_index(drop=True)
    assert len(wm_response) == (n_blocks * n_supertrials), "Check your data length!"
    
    acc_order = accuracy_with_order(wm_response, [i.upper() for i in shown_letters])
    assert len(acc_order) == n_blocks * n_supertrials, "Check your data length!"
    
    acc_norder = accuracy_without_order(wm_response, [i.upper() for i in shown_letters])
    assert len(acc_norder) == n_blocks * n_supertrials, "Check your data length!"
    
    acc_fl = data["fl_accuracy"].dropna().reset_index(drop=True)
    assert len(acc_fl) == (n_blocks * n_supertrials * n_flanker), "Check your data length!"

    blockAcc_wm = data["supertrial_blockAcc_wm"].dropna().reset_index(drop=True)
    assert len(blockAcc_wm) == n_blocks, "Check your data length!"

    fl_supertrial_acc = [np.mean(i) for i in chunk_list(list(acc_fl), 8)]
    assert len(fl_supertrial_acc) == (n_blocks * n_supertrials), "Check your data length!"
    
    blockAcc_fl = [np.mean(i) for i in chunk_list(list(fl_supertrial_acc), 6)]
    assert len(blockAcc_fl) == n_blocks, "Check your data length!"
    
    fl_stim = data["flanker_stim"].dropna().reset_index(drop=True)
    assert len(fl_stim) == (n_blocks * n_supertrials * n_flanker), "Check your data length!"

    for i, key in enumerate(list(data["task_stim_keyResp.keys"])):
        if (pd.isnull(key) and pd.notnull(list(data["fl_accuracy"])[i])) :
            data["task_stim_keyResp.rt"].iloc[i] = "None"      
    
    congruent = data["congruent"].dropna().reset_index(drop=True)
    assert len(congruent) == (n_blocks * n_supertrials * n_flanker), "Check your data length!"

    #print("Accuracy check...")
    if np.mean(acc_norder) < 0.7 or np.mean(acc_fl) < 0.7:
        print ("Low accuracy!")
        print("WM accuracy: {}\nFlanker accuracy: {}".format(round(np.mean(acc_norder), 4), round(np.mean(acc_fl), 4)))
    else:
        print ("All accuracies > 0.7")
        print("WM accuracy: {}\nFlanker accuracy: {}".format(round(np.mean(acc_norder), 4), round(np.mean(acc_fl), 4)))
    print("")

    # rt_trial = data["task_stim_keyResp.rt"].dropna().reset_index(drop=True)
# rt_trial = [i.split(",")for i in list(rt_trial)]
# rt_trial[0]
    
    wm_acc_data = pd.DataFrame({
    "acc_order": acc_order,
    "acc_norder": acc_norder,
    "acc_fl": fl_supertrial_acc,
    "wm_load": ([4]*n_supertrials + [5]*n_supertrials) * (int(n_blocks/2))
    })

    fl_error_wm_acc_norder_1_4 = wm_acc_data[(wm_acc_data["acc_fl"] == 0.875) & (wm_acc_data["wm_load"] == 4)].acc_norder.mean()
    fl_error_wm_acc_norder_1_5 = wm_acc_data[(wm_acc_data["acc_fl"] == 0.875) & (wm_acc_data["wm_load"] == 5)].acc_norder.mean()
    fl_error_wm_acc_norder_0_4 = wm_acc_data[(wm_acc_data["acc_fl"] == 1) & (wm_acc_data["wm_load"] == 4)].acc_norder.mean()
    fl_error_wm_acc_norder_0_5 = wm_acc_data[(wm_acc_data["acc_fl"] == 1) & (wm_acc_data["wm_load"] == 5)].acc_norder.mean()
    fl_error_wm_acc_norder_many_4 = wm_acc_data[(wm_acc_data["acc_fl"] < 0.875) & (wm_acc_data["wm_load"] == 4)].acc_norder.mean()
    fl_error_wm_acc_norder_many_5 = wm_acc_data[(wm_acc_data["acc_fl"] < 0.875) & (wm_acc_data["wm_load"] == 5)].acc_norder.mean()
    
    acc_order_4 = wm_acc_data[wm_acc_data["wm_load"] == 4].acc_order.mean()
    acc_order_5 = wm_acc_data[wm_acc_data["wm_load"] == 5].acc_order.mean()
    acc_norder_4 = wm_acc_data[wm_acc_data["wm_load"] == 4].acc_norder.mean()
    acc_norder_5 = wm_acc_data[wm_acc_data["wm_load"] == 5].acc_norder.mean()
    
    
    rt_data = pd.DataFrame({"wm_load": ([4]*(n_flanker*n_supertrials) + [5]*(n_flanker*n_supertrials)) * (int(n_blocks/2)),
        "rt": list(pd.concat([data["flanker_stim"].dropna(), data["task_stim_keyResp.rt"]], axis=1).dropna()["task_stim_keyResp.rt"]),
                            "congruent": congruent,
                            "acc": acc_fl}
                          )
    
    assert len(rt_data) == (n_blocks * n_supertrials * n_flanker), "Check your data length!"
    
    rt_data["rt"] = convert_strings_to_float_lists(rt_data["rt"])
    rt_data["rt"] = ["N" if i == "None" else i for i in rt_data["rt"]]
    
    rt_data["rt_m"] = [i[0] for i in rt_data["rt"]]
    rt_data["rt"] = [i[0] if len(i) == 1 else "M" for i in rt_data["rt"]]
    
    rt_data["rt"] = [i * 1000 if type(i) == float else i for i in rt_data["rt"]]
    rt_data["rt_m"] = [i * 1000 if type(i) == float else i for i in rt_data["rt_m"]]
    
    rt_data['rt_clean'] = pd.to_numeric(rt_data['rt'], errors='coerce')
    rt_data['rt_m'] = pd.to_numeric(rt_data['rt_m'], errors='coerce')
    
    assert len(rt_data) == (n_blocks * n_supertrials * n_flanker), "Check your data length!"
    
    mean_fl_acc_4 = round(rt_data[(rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 4)].acc.mean(), 2)
    mean_fl_acc_5 = round(rt_data[(rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 5)].acc.mean(), 2)
    acc_con_4 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 4)].acc.mean(), 2)
    acc_con_5 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 5)].acc.mean(), 2)
    acc_incon_4 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 4)].acc.mean(), 2)
    acc_incon_5 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["wm_load"] == 5)].acc.mean(), 2)
    rt_con_corr_4 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 1) & (rt_data["wm_load"] == 4)].rt_m.mean(), 4)
    rt_con_corr_5 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 1) & (rt_data["wm_load"] == 5)].rt_m.mean(), 4)
    rt_incon_corr_4 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 1) & (rt_data["wm_load"] == 4)].rt_m.mean(), 4)
    rt_incon_corr_5 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 1) & (rt_data["wm_load"] == 5)].rt_m.mean(), 4)
    rt_con_incorr_4 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 0) & (rt_data["wm_load"] == 4)].rt_m.mean(), 4)
    rt_con_incorr_5 = round(rt_data[(rt_data["congruent"] == 1) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 0) & (rt_data["wm_load"] == 5)].rt_m.mean(), 4)
    rt_incon_incorr_4 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 0) & (rt_data["wm_load"] == 4)].rt_m.mean(), 4)
    rt_incon_incorr_5 = round(rt_data[(rt_data["congruent"] == 0) & (rt_data["rt_m"] > 150) & (rt_data["acc"] == 0) & (rt_data["wm_load"] == 5)].rt_m.mean(), 4)

    # rt_data["pes"] = np.nan
    # rt_data["pea"] = np.nan
    # for i in range(len(rt_data)-1):
    #     if rt_data["acc"][i] < 1:
    #         pes = rt_data["rt_clean"][i+1] - rt_data["rt_clean"][i]
    #         rt_data["pes"][i+1] = pes
    # pes = rt_data["pes"].mean()

    #print("sub: {}".format(sub))
    # print("congruent RT: {} ms".format(rt_con))
    # print("congruent acc: {}".format(acc_con))
    # print("incongruent RT: {} ms".format(rt_incon))
    # print("incongruent acc: {}".format(acc_incon))
    print("")
    print("-----------")

    chi_data = pd.DataFrame({
    "wm_error":[1 if i<1 else 0 for i in acc_order],
    "fl_error":[1 if i<1 else 0 for i in fl_supertrial_acc]
    })
    chi2, p, dof, expected = stats.chi2_contingency(pd.crosstab(chi_data["wm_error"], chi_data["fl_error"]))
    crosstab = pd.crosstab(chi_data["wm_error"], chi_data["fl_error"]).stack()
    wm_err0_fl_err0 = crosstab[0,0] # wm-err 0 fl-err 0
    wm_err0_fl_err1 = crosstab[0,1] # wm-err 0 fl-err 1
    wm_err1_fl_err0 = crosstab[1,0] # wm-err 1 fl-err 0
    wm_err1_fl_err1 = crosstab[1,1] # wm-err 1 fl-err 1

    intersection = dict({
    "shown_letters":[],
    "response":[],
    "fl_middle":[],
    "shown_x_flanker":[],
    "response_x_flanker":[],
    "difference":[],
    "acc_order":[],
    "acc_norder":[],
    "acc_fl":[],
    "congruent":[],
    "acc":[],
    })
    for i in range(n_supertrials * n_blocks):
        fl_middle= "".join([fl[2] for fl in chunk_list(list(fl_stim), 8)[i]])
        congr = [fl for fl in chunk_list(list(congruent), 8)[i]]
        accuracy = [ac for ac in chunk_list(list(acc_fl), 8)[i]]
        intersection["shown_letters"].append(shown_letters[i])
        intersection["response"].append(wm_response[i])
        intersection["fl_middle"].append(fl_middle)
        intersection["shown_x_flanker"].append(set(shown_letters[i].upper()).intersection(fl_middle.upper()))
        intersection["response_x_flanker"].append(set(wm_response[i].upper()).intersection(fl_middle.upper()))
        intersection["difference"].append(
            set(wm_response[i].upper()).intersection(fl_middle.upper()).\
            difference(set(shown_letters[i].upper()).intersection(fl_middle.upper()))
        )
        intersection["acc_order"].append(acc_order[i])
        intersection["acc_norder"].append(acc_norder[i])
        intersection["acc_fl"].append(fl_supertrial_acc[i])
        intersection["congruent"].append(congr)
        intersection["acc"].append(accuracy)

    intersec_data = pd.DataFrame(intersection)
    intersec_data["wm_load"] = ([4]*n_supertrials + [5]*n_supertrials) * (int(n_blocks/2))
    intersec_data["diff_con"] = np.nan
    intersec_data["diff_incon"] = np.nan
    for i, diff in enumerate(intersec_data["difference"]): #correct
        con_ct = 0
        incon_ct = 0
        for letter in diff:
            ind = intersec_data["fl_middle"][i].upper().index(letter)
            if (intersec_data["congruent"][i][ind] == 1) and (intersec_data["acc"][i][ind] == 1):
                con_ct +=1
            elif (intersec_data["congruent"][i][ind] == 0) and (intersec_data["acc"][i][ind] == 1):
                incon_ct +=1
        intersec_data["diff_con"][i] = con_ct
        intersec_data["diff_incon"][i] = incon_ct

    intersec_data["len_difference"] = [len(i) for i in intersec_data["difference"]]
    intersec_data["1h_difference"] = [1 if len(i)>0 else 0 for i in intersec_data["difference"]]
    mean_letters_from_fl_err_4 = intersec_data[(intersec_data["acc_fl"] < 1) & (intersec_data["wm_load"] == 4)].len_difference.mean()
    mean_letters_from_fl_err_5 = intersec_data[(intersec_data["acc_fl"] < 1) & (intersec_data["wm_load"] == 5)].len_difference.mean()
    mean_letters_from_fl_corr_4 = intersec_data[(intersec_data["acc_fl"] == 1) & (intersec_data["wm_load"] == 4)].len_difference.mean()
    mean_letters_from_fl_corr_5 = intersec_data[(intersec_data["acc_fl"] == 1) & (intersec_data["wm_load"] == 5)].len_difference.mean()

    intersec_subset = intersec_data[(intersec_data["difference"] != set()) & (intersec_data["acc_fl"] == 0.875)]
    cnt = []
    for i in range(len(intersec_subset)):
        ind_err = intersec_subset.acc.reset_index(drop=True)[i].index(0)
        inds_letter = [intersec_subset.fl_middle.reset_index(drop=True)[i].upper().index(letter) for letter in intersec_subset.difference.reset_index(drop=True)[i]]
        if any([ind_err == il for il in inds_letter]):
            cnt.append(1)
        else:
            cnt.append(0)
    intersec_subset["cnt"] = cnt
    intersec_subset["cnt_n"] = len(cnt)
    cnt_mean = intersec_subset["cnt"].mean()
    subsets.append(intersec_subset)

    #if np.mean(acc_norder) > 0.7 and np.mean(acc_fl) > 0.7:
    wme_dict["sub"].append(sub)
    wme_dict["acc_fl"].append(np.mean(acc_fl))
    wme_dict["acc_norder"].append(np.mean(acc_norder))
    wme_dict["4-1_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_1_4)
    wme_dict["5-1_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_1_5)
    wme_dict["4-0_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_0_4)
    wme_dict["5-0_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_0_5)
    wme_dict["4-many_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_many_4)
    wme_dict["5-many_fl_error_wm_acc_norder"].append(fl_error_wm_acc_norder_many_5)
    wme_dict["4-acc_con"].append(acc_con_4)
    wme_dict["5-acc_con"].append(acc_con_5)
    wme_dict["4-acc_incon"].append(acc_incon_4)
    wme_dict["5-acc_incon"].append(acc_incon_5)
    wme_dict["4-acc_fl"].append(mean_fl_acc_4)
    wme_dict["5-acc_fl"].append(mean_fl_acc_5)
    wme_dict["1_fl_error_trials"].append(round(len(intersec_data[intersec_data["acc_fl"]==0.875])/len(intersec_data[intersec_data["acc_fl"]<1]), 2))
    wme_dict["4-rt_con_corr"].append(rt_con_corr_4)
    wme_dict["5-rt_con_corr"].append(rt_con_corr_5)
    wme_dict["4-rt_incon_corr"].append(rt_incon_corr_4)
    wme_dict["5-rt_incon_corr"].append(rt_incon_corr_5)
    wme_dict["4-rt_con_incorr"].append(rt_con_incorr_4)
    wme_dict["5-rt_con_incorr"].append(rt_con_incorr_5)
    wme_dict["4-rt_incon_incorr"].append(rt_incon_incorr_4)
    wme_dict["5-rt_incon_incorr"].append(rt_incon_incorr_5)
    #wme_dict["pes"].append(pes)
    wme_dict["4-acc_order"].append(acc_order_4)
    wme_dict["5-acc_order"].append(acc_order_5)
    wme_dict["4-acc_norder"].append(acc_norder_4)
    wme_dict["5-acc_norder"].append(acc_norder_5)
    wme_dict["wm_err0_fl_err0"].append(wm_err0_fl_err0)
    wme_dict["wm_err0_fl_err1"].append(wm_err0_fl_err1)
    wme_dict["wm_err1_fl_err0"].append(wm_err1_fl_err0)
    wme_dict["wm_err1_fl_err1"].append(wm_err1_fl_err1)
    wme_dict["chi2"].append(p)
    wme_dict["diff_con"].append(intersec_data["diff_con"].mean())
    wme_dict["diff_incon"].append(intersec_data["diff_incon"].mean())
    wme_dict["4-mean_letters_from_fl_err"].append(mean_letters_from_fl_err_4)
    wme_dict["5-mean_letters_from_fl_err"].append(mean_letters_from_fl_err_5)
    wme_dict["4-mean_letters_from_fl_corr"].append(mean_letters_from_fl_corr_4)
    wme_dict["5-mean_letters_from_fl_corr"].append(mean_letters_from_fl_corr_5)
    wme_dict["cnt_mean"].append(cnt_mean)
    wme_dict["cnt_n"].append(intersec_subset["cnt_n"].mean())
    wme_dict["hand"].append(counterbalance[0])
    wme_dict["repeat"].append(counterbalance[1])

    fl_trials = pd.DataFrame(
        {"target": [i[2] for i in fl_stim],
        "congruent": congruent,
        "acc": acc_fl,
        }
    )
    labels = []
    for i in range(len(fl_trials)):
        if fl_trials["congruent"][i] == 0 and fl_trials["acc"][i] == 0: # incon-error
            labels.append("incon-error")
        elif fl_trials["congruent"][i] == 0 and fl_trials["acc"][i] == 1: # incon-correct
            labels.append("incon-correct")
        else:
            labels.append("NA")
    fl_trials["label"] = labels
    
    post_labels = []
    post_labels.append("NA")
    for i in range(1, len(fl_trials)):
        if fl_trials["label"][i-1] == "incon-correct":
            post_labels.append("post incon-correct")
        elif fl_trials["label"][i-1] == "incon-error":
            post_labels.append("post incon-error")
        else:
            post_labels.append("NA")
    fl_trials["post_label"] = post_labels
    fl_trials["target"] = [i.upper() for i in fl_trials["target"]]
    
    recalls = []
    for i in wm_response:
        recalls += [i] * 8
    
    fl_trials["recall"] = recalls
    fl_trials["load"] = ([4]*n_flanker*n_supertrials + [5]*n_flanker*n_supertrials) * (int(n_blocks/2))
    
    appeared = []
    for i in range(len(fl_trials)):
        if fl_trials["target"][i] in fl_trials["recall"][i]:
            appeared.append(True)
        else:
            appeared.append(False)

    fl_trials["appeared"] = appeared

    ratio_incon_correct = len((fl_trials[(fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == False)]))

    ratio_incon_correct_4 = len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == False)]))

    ratio_incon_correct_5 = len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["label"] == "incon-correct") & (fl_trials["appeared"] == False)]))
    
    ratio_post_incon_correct = len((fl_trials[(fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == False)]))

    ratio_post_incon_correct_4 = len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == False)]))

    ratio_post_incon_correct_5 = len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["post_label"] == "post incon-correct") & (fl_trials["appeared"] == False)]))
    
    ratio_incon_error = len((fl_trials[(fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == False)]))

    ratio_incon_error_4 = len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == False)]))

    ratio_incon_error_5 = len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["label"] == "incon-error") & (fl_trials["appeared"] == False)]))
    
    ratio_post_incon_error = len((fl_trials[(fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == False)]))

    ratio_post_incon_error_4 = len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 4) & (fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == False)]))

    ratio_post_incon_error_5 = len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == True)]))/\
    len((fl_trials[(fl_trials["load"] == 5) & (fl_trials["post_label"] == "post incon-error") & (fl_trials["appeared"] == False)]))

    wme_dict["ratio_incon_correct"].append(ratio_incon_correct)
    wme_dict["4-ratio_incon_correct"].append(ratio_incon_correct_4)
    wme_dict["5-ratio_incon_correct"].append(ratio_incon_correct_5)
    
    wme_dict["ratio_post_incon_correct"].append(ratio_post_incon_correct)
    wme_dict["4-ratio_post_incon_correct"].append(ratio_post_incon_correct_4)
    wme_dict["5-ratio_post_incon_correct"].append(ratio_post_incon_correct_5)

    wme_dict["ratio_incon_error"].append(ratio_incon_error)
    wme_dict["4-ratio_incon_error"].append(ratio_incon_error_4)
    wme_dict["5-ratio_incon_error"].append(ratio_incon_error_5)
    
    wme_dict["ratio_post_incon_error"].append(ratio_post_incon_error)
    wme_dict["4-ratio_post_incon_error"].append(ratio_post_incon_error_4)
    wme_dict["5-ratio_post_incon_error"].append(ratio_post_incon_error_5)

    big_list = [i[2].upper() for i in fl_stim]
    chunk_size = 8
    result = chunk_list(big_list, chunk_size)
    result = ["".join(i) for i in result]
    
    list1 = ["".join(i) for i in result]
    list2 = [i.upper() for i in shown_letters]
    repeat_cnt = 0
    not_repeat_cnt = 0
    for string1, string2 in zip(list1, list2):
        if has_common_letter(string1, string2):
            repeat_cnt +=1
        else:
            not_repeat_cnt +=1

    wme_dict["repeat_cnt"].append(repeat_cnt)
    wme_dict["not_repeat_cnt"].append(not_repeat_cnt)

In [ ]:
wme_data = pd.DataFrame(wme_dict)
wme_data = wme_data.round(2)
wme_data = wme_data[(wme_data["acc_fl"]>0.7) & (wme_data["4-acc_norder"]>0.7)].reset_index(drop=True)
wme_data